Author: Liu Chang

Description: 

This notebook will demonstrate CNN+RNN hybrid model performance 

Date:
Week 9

In [1]:
import os.path as path
import pickle
import torch
from torch import nn

data_path='data/'

with open(path.join(data_path,'data_X'),'rb') as f:
    X=pickle.load(f)

with open(path.join(data_path,'data_Y_int'),'rb') as f:
    Y=pickle.load(f)

len_train_X=int(len(X)*0.8)

print(len(X))
import numpy as np
train_X=np.array(X[:len_train_X])
train_Y=np.array(Y[:len_train_X])
test_X=np.array(X[len_train_X:])
test_Y=np.array(Y[len_train_X:])

4572


In [2]:
labels_count=len(set(test_Y))
print(labels_count)

5


In [3]:
def unpack(x,max_strock,max_len):
    x_new=torch.zeros(torch.Size([x.shape[0],max_strock,3,max_len]))
    for i,item in enumerate(x):
        for j,strock in enumerate(item):
            #print(strock)
            
            #print(len(strock[0]))
            strock=torch.Tensor(strock)
            #print(x_new[i,j,0,:len(strock[0])].shape)
            #print(strock[0].shape)
            x_new[i,j,0,:len(strock[0])]=strock[0]
            x_new[i,j,1,:len(strock[0])]=strock[1]
            x_new[i,j,2,:len(strock[0])]=0
            x_new[i,j,2,len(strock[0])-1]=1
    return x_new

def get_max_len(x):
    max_len=0
    max_strock=0
    for i,item in enumerate(x):
        max_strock=max(max_strock,len(item))
        #print(max_strock)
        for j,strock in enumerate(item):
            max_len=max(max_len,len(strock[0])) 
    return max_strock,max_len

max_strock,max_len=get_max_len(X)
print(max_strock,max_len)
train_X=unpack(train_X,max_strock,max_len)
test_X=unpack(test_X,max_strock,max_len)

47 123


In [4]:
print(train_X.shape)

torch.Size([3657, 47, 3, 123])


In [5]:
def eval_on_test_set(test_data,test_label,num_layers,bs,hidden_size,net,criterion):
    
    running_loss=0
    num_batches=0  
    
    correct = 0
    total = 0
       
    # set the initial h and c to be the zero vector

    # send them to the gpu    
    # h=h.to(device)
    # c=c.to(device) len(test_X)
    
    for count in range( 100) :
               
        minibatch_data =  test_data[ count ] # bs*seq_len*2
        minibatch_data = torch.Tensor(minibatch_data)
        minibatch_label = test_label[ count]    
        minibatch_label = torch.Tensor([minibatch_label.item()])
        
        # minibatch_data=minibatch_data.to(device)
        # minibatch_label=minibatch_label.to(device)
                                  
        scores  = net( minibatch_data)
         
        
        loss = criterion(  scores ,  minibatch_label )    
            
        running_loss += loss.item()
        num_batches += 1        
        
        _, predicted = torch.max(scores.data, 1)
        total += minibatch_label.size(0)
        correct += (predicted == minibatch_label).sum().item()
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
    print ('Test accuracy:{}%'.format(100 * correct / total))

In [6]:
class CNN(nn.Module):
    def __init__(self,output_size):
        super(CNN, self).__init__()
        # self.conv1 = nn.Sequential(
        #     nn.Conv1d(
        #         in_channels=3,      # input height
        #         out_channels=6,    # n_filters
        #         kernel_size=2      # filter size
        #     ),
        #     nn.ReLU(),    # activation
        # )
        # self.conv2 = nn.Sequential(
        #     nn.Conv1d(6,9, 4),
        #     nn.ReLU(),  # activation
        # )
        self.conv3 = nn.Sequential(
            nn.Conv1d(3,32,20),
            nn.ReLU(),  # activation
        )
        self.out = nn.Linear(12 * (max_len-(2+4+6-3)), output_size)
        self.out2 = nn.Linear(3*(max_len), output_size)

    def forward(self, x):
        # x = self.conv1(x)
        # x = self.conv2(x)
        # x=self.conv3(x)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        # output = self.out(x)
        output=self.out2(x)
        return output
    
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.middle_size=middle_size=30
        self.gru=nn.GRU(input_size=middle_size,hidden_size=5,num_layers=1,bias=False)
        # self.gru=nn.GRU(input_size=3,hidden_size=5,num_layers=1,bias=True)
        self.cnn=CNN(output_size=middle_size)
        self.out=nn.Softmax(dim=-1)
    def forward(self, x):
        x=x.view(-1,3,max_len)
        
        #print(x.shape)
        x = self.cnn(x)
        x=x.view(-1,max_strock,self.middle_size).transpose(0,1)
        output,_ = self.gru(x)
        #print(output.shape)
        output = self.out(output[-1])
        return output

In [7]:
# x_rand=torch.randn(torch.Size([13,max_strock,3,max_len]))
# print(x_rand.shape)
#model=CRNN()
# print("---")
# y_rand=model(x_rand)[-1]
# print(y_rand.shape)

In [30]:
import torch.optim as optim

model=CRNN().cuda()
batch_size=128
loss=torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
EPOCH=256
train_Y=torch.LongTensor(train_Y)
test_Y=torch.LongTensor(test_Y)
for i in range(EPOCH):
    total_loss=0
    cur_len=0
    while cur_len<train_X.shape[0]:
        model.zero_grad()
        minibatch_X=train_X[cur_len:cur_len+batch_size].cuda()
        #print(minibatch_X.shape)
        minibatch_Y=train_Y[cur_len:cur_len+batch_size].cuda()
        y_predict=model(minibatch_X)
        #print(y_predict.shape)
        #print(y_predict.shape,minibatch_Y.shape)
        #print(loss,y_predict,minibatch_Y)
        output_loss=loss(y_predict,minibatch_Y)
        output_loss.backward()
        optimizer.step()
        cur_len+=batch_size
        total_loss+=output_loss.item()
        #print("Processing: {}/{}, loss={}".format(cur_len,train_X.shape[0],output_loss.item()))
    if i%20==0:
        print("EPOCH {}/{}, loss={}".format(i,EPOCH,total_loss))

EPOCH 0/256, loss=46.62959015369415
EPOCH 20/256, loss=45.74260628223419
EPOCH 40/256, loss=45.073907136917114
EPOCH 60/256, loss=44.46343731880188
EPOCH 80/256, loss=44.193055629730225
EPOCH 100/256, loss=43.77127933502197
EPOCH 120/256, loss=43.28636872768402
EPOCH 140/256, loss=43.11358892917633
EPOCH 160/256, loss=42.850329756736755
EPOCH 180/256, loss=42.8080815076828
EPOCH 200/256, loss=42.78309988975525
EPOCH 220/256, loss=42.63023924827576
EPOCH 240/256, loss=42.86384308338165


In [32]:
cur_len=0
acc=0
while cur_len<test_X.shape[0]:
    model.zero_grad()
    minibatch_X=test_X[cur_len:cur_len+batch_size].cuda()
    #print(minibatch_X.shape)
    minibatch_Y=test_Y[cur_len:cur_len+batch_size].cuda()
    y_predict=model(minibatch_X)
    y_predict=torch.argmax(y_predict,dim=1)
    #print(y_predict.shape)
    cur_len+=batch_size
    acc+=(y_predict==minibatch_Y).sum().item()
print("Accuracy on test set: {}".format(acc/cur_len))
    #print("Processing: {}/{}, loss={}".format(cur_len,train_X.shape[0],output_loss.item()))

Accuracy on test set: 0.4033203125
